In [ ]:
import os, math, time, pickle, subprocess
from importlib import reload
from collections import OrderedDict
import numpy as np
import pandas as pd
pd.set_option('display.width', 100)
import epitopepredict as ep
from epitopepredict import base, sequtils, tepitope, plotting, utilities, peptutils, mhclearn, analysis, web
from IPython.display import display, HTML, Image
%matplotlib inline
import matplotlib as mpl
import pylab as plt
from os import listdir
from os.path import isfile, join

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
home = os.path.expanduser("~/Documentos/Master_work")
save = input('Insira o path: ')
# save = os.path.dirname("Documents/Master_work/Sars2/)
savepath=os.path.join(home, save)
print(savepath)

In [ ]:
folder = input('Insira o nome da pasta: ')

try:
#     parent_dir = '/home/pedro/Documentos/Master_work/resultados_sars2/S_RefSeq/'
    os.chdir(savepath)
    origin = os.path.join(savepath, folder)
    os.mkdir(origin)
    print('Criado pasta %s: %s' %(folder,origin))
except OSError:
    print('Pasta já existe no diretório')
    pass

In [ ]:
a= os.getcwd()
print(a)

In [ ]:
predictors = ['iedbmhc1','netmhcpan','mhcflurry','iedbmhc2','netmhciipan']

try:
    for i in predictors:
        files = os.path.join(origin,i)
        try: 
            os.mkdir(files) 
        except OSError: 
            print('pasta %s já existe' %(i))
except OSError:
    pass


iedbmhcii = ['recommended','netmhciipan_el']

try:
    for j in iedbmhcii:
        files2 = os.path.join(origin,'iedbmhc2',j)
        try: 
            os.mkdir(files2) 
        except OSError: 
            print('pasta %s já existe' %(j))
except OSError:
    pass

## Alleles

In [ ]:
#get preset alleles
m2_alleles = ep.get_preset_alleles('mhc2_IEDB')
m1_alleles = ep.get_preset_alleles('mhc1_IEDB')
print (m1_alleles)
print (m2_alleles)

## Sequence

In [ ]:
homeseq = os.path.expanduser("~")
saveseq = os.path.dirname("Sequences/")
savepathseq=os.path.join(homeseq, saveseq)

onlyfiles = [f for f in listdir(savepathseq) if isfile(join(savepathseq, f))]
print('lista de sequêmcias na pasta Sequences:\n','\033[1m'+'\033[31m'+ ', '.join(onlyfiles)+'\033[0;0m','\n',savepathseq)

nameseq = input('Insira o nome da sequência: ')

fastafile = os.path.join(savepathseq, nameseq)
print(fastafile)
prot = ep.fasta_to_dataframe(fastafile, header_sep=None, key='locus_tag', seqkey='translation')
locus_tag = list([prot.locus_tag[0]])
display(prot[:20])

In [ ]:
iedbmhc1_path = os.path.join(origin,'iedbmhc1')
print(iedbmhc1_path)
netmhcpan_path = os.path.join(origin,'netmhcpan')
mhcflurry_path = os.path.join(origin,'mhcflurry')
recommended_path = os.path.join(origin,'iedbmhc2','recommended')
netmhciipan_el_path = os.path.join(origin,'iedbmhc2','netmhciipan_el')
netmhciipan_path = os.path.join(origin,'netmhciipan')

In [ ]:
import multiprocessing as mp
maxcpu = mp.cpu_count()
print('cpus -> ', maxcpu)

# MHCI

### MHCflurry

In [ ]:
reload(base)
mhcflurry = base.get_predictor('mhcflurry')
mhcflurry.predict_sequences(prot,path=mhcflurry_path,alleles=m1_alleles, length=9, verbose=True, threads=8)
mhcflurry.load(path=mhcflurry_path)

### IEDB MHCI

In [ ]:
reload(base)
iedb = base.get_predictor('iedbmhc1')
iedb.predict_sequences(prot,path=iedbmhc1_path,alleles=m1_alleles, length=9,method='IEDB_recommended',verbose=True,threads=8)
iedb.load(path=iedbmhc1_path)

### NetMHCpan

In [ ]:
reload(base)
netmhcpan = base.get_predictor('netmhcpan')
netmhcpan.predict_sequences(prot,path=netmhcpan_path,alleles=m1_alleles, length=9,method='IEDB_recommended',verbose=True,threads=8)
netmhcpan.load(path=netmhcpan_path)

# MHC II

### IEDB MHCII Recommended

In [ ]:
reload(base)
iedbii = base.get_predictor('iedbmhc2')
iedbii.predict_sequences(prot,path=recommended_path,alleles=m2_alleles, length=15,method='IEDB_recommended',verbose=True,threads=8)
iedbii.load(path=recommended_path)

### IEDB MHCII NetMHCIIpan

In [ ]:
reload(base)
iedbiin = base.get_predictor('iedbmhc2')
iedbiin.predict_sequences(prot,path=netmhciipan_el_path,alleles=m2_alleles, length=15,method='netmhciipan_el',verbose=True,threads=8)
iedbiin.load(path=netmhciipan_el_path)

### NetMHCIIpan

In [ ]:
reload(base)
netmhciipan = base.get_predictor('netmhciipan')
netmhciipan.predict_sequences(prot,path=netmhciipan_path,alleles=m2_alleles, length=15,verbose=True,threads=8)
netmhciipan.load(path=netmhciipan_path)